In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread

# **Práctica Dirigida: Calibrando una imagen para medición de distancia**

In [ ]:
!gdown --id 1DpAa_ZegoM4bjcTbhel0jyNkhg5bHfr4

In [ ]:
filename = 'imagen.png'
im = imread(filename, as_gray=True)
plt.imshow(im, cmap='gray')

In [ ]:
#@title Default title text { run: "auto" }
th = 0.36 #@param {type:"slider", min:0, max:1, step:0.01}
im_th = (im>th)*1.0 
plt.imshow(im_th, cmap='gray')

In [ ]:
im.shape

In [ ]:
#@title Default title text { run: "auto" }
porcentaje = 0.04 #@param {type:"slider", min:0, max:1, step:0.01}
top_n_pixels = int(im.shape[0]*porcentaje) #Vamos a tomar el 2%  superior de la imagen
plt.imshow(im[:top_n_pixels, :], cmap='gray')

### Generar Templates

In [ ]:
def generate_pattern(h, w, pixel_spacing=0.1, offset=0):
  x=np.linspace(0, 2*np.pi, w)
  y=np.sin(x/pixel_spacing*2*np.pi+offset)
  return np.repeat(y, h).reshape(h,w,order='F')

Probar el método

In [ ]:
plt.figure(figsize=(20,10))
plt.imshow(generate_pattern(100, 1000, pixel_spacing=0.2, offset=8), cmap='gray')

### Implementar Similaridad de Coseno

Recuerde la definición: $cos(a, b)= \frac{a . b}{\|a\| \|b\|}$ donde $a . b$ es el producto interno de los vectores $a$ y $b$ 

In [ ]:
def cosine(input, template):
  ### Implemente aqui
  #recuerde que tiene a y b seran matrices, pero con .ravel() 
  #podra volverlos vectores

  return np.dot(input.ravel(), template.ravel())/(np.linalg.norm(input)*np.linalg.norm(template))

  #######################

Checks:

In [ ]:
cosine(np.array([1,2,3]), np.array([1,2,3]))

In [ ]:
cosine(np.array([1,0]), np.array([0,1]))

### Implementar búsqueda y cálculo de score

In [ ]:
def compute_score(input, ps_search_space, offset_search_space):
  ### Implemente aqui

  values = []
  for pixel_spacing in ps_search_space:
    scores = []
    for offset in offset_search_space:
      template = generate_pattern(input.shape[0], 
                                  input.shape[1], 
                                  pixel_spacing=pixel_spacing, 
                                  offset=offset)
      score = cosine(input, template)
      scores.append(score)
    values.append(np.max(scores))
  
  return np.array(values)


  #######################


### ... finalmente

In [ ]:
ps_search_space = np.linspace(0.05, 0.15, 500)
offset_search_space = np.arange(10)

In [ ]:
scores = compute_score(im_th[:top_n_pixels, :], ps_search_space, offset_search_space)

grafiquemos

In [ ]:
plt.figure()
plt.ylabel("Probability")
plt.xlabel("Pixel Spacing")
plt.plot(ps_search_space, scores)

Calcular el mejor ajuste

In [ ]:
pixel_spacing = ps_search_space[np.argmax(scores)]

In [ ]:
pixel_spacing